In [1]:
import os.path as osp
import argparse

import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, ChebConv  # noqa

In [3]:
dataset = 'Cora'
dataset = Planetoid("./data/", dataset, transform=T.NormalizeFeatures())
data = dataset[0]

Processing...
Done!


In [4]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16, cached=True,
                             normalize=True)
        self.conv2 = GCNConv(16, dataset.num_classes, cached=True,
                             normalize=True)
        # self.conv1 = ChebConv(data.num_features, 16, K=2)
        # self.conv2 = ChebConv(16, data.num_features, K=2)

    def forward(self):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return F.log_softmax(x, dim=1)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam([
    dict(params=model.conv1.parameters(), weight_decay=5e-4),
    dict(params=model.conv2.parameters(), weight_decay=0)
], lr=0.01)  # Only perform weight-decay on first convolution.

In [6]:
def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()

@torch.no_grad()
def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [7]:
data.train_mask

tensor([ True,  True,  True,  ..., False, False, False])

In [10]:
tmp0 = model()

In [11]:
tmp0

tensor([[-1.9494, -1.9476, -1.9470,  ..., -1.9447, -1.9417, -1.9527],
        [-1.9486, -1.9429, -1.9431,  ..., -1.9470, -1.9427, -1.9549],
        [-1.9521, -1.9462, -1.9389,  ..., -1.9443, -1.9534, -1.9483],
        ...,
        [-1.9517, -1.9680, -1.9184,  ..., -1.9582, -1.9476, -1.9318],
        [-1.9480, -1.9480, -1.9445,  ..., -1.9439, -1.9471, -1.9470],
        [-1.9475, -1.9481, -1.9453,  ..., -1.9436, -1.9468, -1.9472]],
       grad_fn=<LogSoftmaxBackward>)

In [12]:
tmp0.size()

torch.Size([2708, 7])